In [1]:
import json
import os

import pandas as pd
import spacy
from joblib import dump
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import RFECV, SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from spacy.pipeline.functions import merge_entities
from tqdm import tqdm

In [2]:
from train_documents_classification_model import *

C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\spacy\util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
DATASET_PATH

'data'

In [4]:
df = load_dataset()

In [5]:
df

,text,label
0,PCAR \n \n \n \n \n \n PART 7 \n \n \n \...,1
1,LIQUOR AMENDMENT (24-HOUR ECONOMY) BILL 2020 \...,1
2,Subordinate Legislation Act 1989 No 146\nNew S...,1
3,GOVERNMENT GAZETTE – DD Month YYYY\nGovernment...,1
4,\nNew South Wales\nCOVID-19 Legislation Amend...,1
...,...,...
401,\nPublic Health Brief \n \nRichard O. Johnson...,0
402,\nPublic Health Brief \n \nRichard O. Johnson...,0
403,"Public Health Brief \n \nRichard O. Johnson, M...",0
404,THE ESTATE AGENTS (AMENDMENT) BILL (5 MAY 2020...,0


In [6]:
train, test = train_test_split(df, stratify=df.label, random_state=123)
text_preprocessing = FunctionTransformer(func=preprocess_text)

mutual_info_selector = SelectKBest(mutual_info_classif, k=10000)
recurse_importance_selector = RFECV(
    estimator=LogisticRegression(penalty="l1", solver="saga"),
    min_features_to_select=20,
    n_jobs=-1,
    verbose=True,
    step=10,
)
classifier = LogisticRegression(penalty="l2")

In [7]:
pipeline = Pipeline(
    [
        (
            "preprocessing",
            Pipeline(
                [
                    ("text_processor", text_preprocessing),
                    ("count_vectorizer", CountVectorizer()),
                ]
            ),
        ),
        (
            "feature_selection",
            Pipeline(
                [
                    ("mutual_info_selector", mutual_info_selector),
                    ("recurse_importance_selector", recurse_importance_selector),
                ]
            ),
        ),
        ("classifier", classifier),
    ]
)

X_train = train.text
y_train = train.label

In [8]:
type(test.text)

pandas.core.series.Series

In [9]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_train)

print("Train acc:", accuracy_score(y_train, y_pred))
X_test = test.text
y_test = test.label

pred_test = pipeline.predict(X_test)

print("Test acc:", accuracy_score(y_test, pred_test))

dump(pipeline, "data/classification_pipeline.joblib")

100%|███████████████████████████████████████████████████████████████████████| 304/304 [00:00<00:00, 315.10it/s]


Fitting estimator with 10000 features.


C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Fitting estimator with 9990 features.


C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


...

C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Karol\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Karol\A

Train acc: 1.0


100%|███████████████████████████████████████████████████████████████████████| 102/102 [00:00<00:00, 393.71it/s]


Test acc: 0.9803921568627451


['data/classification_pipeline.joblib']

In [10]:
y_test.shape

(102,)

In [11]:
from sklearn.metrics import f1_score
print("Test f1:", f1_score(y_test, pred_test))

Test f1: 0.9666666666666666


In [ ]:
import numpy as np

In [ ]:
# np.savetxt('lr_coefs.txt', pipeline.steps[2][1].coef_)

In [ ]:
# dump(pipeline, "data/classification_pipeline.joblib")